In [1]:
import pandas
import numpy as np
from pandas import read_csv
from sklearn.metrics import accuracy_score
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ["preg", "plas", "pres", "skin", "test", "mass", "pedi", "age", "class"]
data = read_csv(url, names = names)

veri = data.values
X = veri[:, 0:8] #Girdi
Y = veri[:, 8] #Çıktı

In [2]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [3]:
# TOPLULUK ÖĞRENMESİ (ENSEMBLE) İLE PERFORMANS GELİŞTİRME
# Birden çok yöntemin sonuçlarını birleştirir. Böylece veri setinde birden fazla model kullanılır ve bunların güçlü yönlerinden faydalanılır.(Knime'daki fusion predictor)
# Sınıflandırma verilerinde kullanılır.
#Knime'daki Fusion predictor düğümünün Python'daki karşılığı --> VotingClassifier fonksiyonu 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier

In [4]:
model1 = RandomForestClassifier(n_estimators = 250)
model2 = RandomForestClassifier(n_estimators = 450)
model3 = GradientBoostingClassifier()
model4 = XGBClassifier()

In [5]:
Oylama_modeli = VotingClassifier(estimators = [("rf1", model1), ("rf2", model2), ("gbc", model3), ("xgb", model4)], voting = "soft")
#Modelleri birleştirdik.
Oylama_modeli.fit(x_train, y_train)
y_pred = Oylama_modeli.predict(x_test)

print("Test verisi için doğruluk: ", accuracy_score(y_pred, y_test))
print("Eğitim verisi için doğruluk: ", Oylama_modeli.score(x_train, y_train))

Test verisi için doğruluk:  0.7835497835497836
Eğitim verisi için doğruluk:  0.9851024208566108


In [6]:
Oylama_modeli2 = VotingClassifier(estimators = [("rf1", model1), ("rf2", model2), ("gbc", model3), ("xgb", model4)], weights = [1,3,2,4], voting = "soft")
#Modelleri birleştirdik ve modellere farklı ağırlık değerleri atayarak önem derecelerini belirledik.

Oylama_modeli2.fit(x_train, y_train)
y_pred = Oylama_modeli2.predict(x_test)

print("Test verisi için doğruluk: ", accuracy_score(y_pred, y_test))
print("Eğitim verisi için doğruluk: ", Oylama_modeli2.score(x_train, y_train))

Test verisi için doğruluk:  0.7878787878787878
Eğitim verisi için doğruluk:  0.9795158286778398


In [7]:
# PARAMETRE OPTİMİZASYONU
import pandas
import numpy as np
from pandas import read_csv
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [8]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ["preg", "plas", "pres", "skin", "test", "mass", "pedi", "age", "class"]
data = read_csv(url, names = names)

veri = data.values
X = veri[:, 0:8] #Girdi
Y = veri[:, 8] #Çıktı

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [10]:
sc = StandardScaler() #Veriyi normalize ettik.
X_train_std = sc.fit_transform(x_train)
X_test_std = sc.fit_transform(x_test)

In [11]:
# 1. GRID SEARCH  Bir parametre uzayı belirlenir ve model o uzaydaki bütün olasılıkları deneyerek en iyi parametre değerlerini bulur.
# (Knime'daki parametre optimizasyon düğümünün python'daki karşılığıdır.)
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter = 3000, solver = "lbfgs").fit(X_train_std, y_train)
y_pred = clf.predict(X_test_std)
print("Test Performansları;")
print("Doğruluk: ", (accuracy_score(y_test, y_pred)))
print("Precision: " + str(precision_score(y_test, y_pred)))
print("Recall: " + str(recall_score(y_test, y_pred)))
print("F1 puanı: " + str(f1_score(y_test, y_pred)))
print("\n")
print("Sınıflandırma Tablosu: \n" + str(confusion_matrix(y_test, y_pred)))

Test Performansları;
Doğruluk:  0.7705627705627706
Precision: 0.6666666666666666
Recall: 0.5675675675675675
F1 puanı: 0.6131386861313869


Sınıflandırma Tablosu: 
[[136  21]
 [ 32  42]]


In [12]:
# Regresyonda 2 temel gösterge kullanılabilir: Birincisi ceza fonksiyonu (L1 veya L2), bir diğeri ise düzenlileştirme parametresi->lambda( C parametresi)
grid_değerleri = {"C": [0.0001, 0.009, 0.01, 0.05, 0.09, 1, 5, 10, 25, 100]}
clf = LogisticRegression(max_iter = 3000, solver = "lbfgs")
grid_model = GridSearchCV(clf, param_grid = grid_değerleri, cv = 10, scoring = "recall", verbose = 1) # cv = 10 herbir parametre değeri için 10 deneme yapar.(Grid değerlerindeki parametreler)
#Parametre optimizasyonunu gerçekleştirecek fonksiyon --> GridSearch fonkisoynu,  CV = Cross validation(Çapraz doğrulama)
grid_model.fit(X_train_std, y_train)
# verbose ekrana bilgilendirme mesajı yazdırır.
#10 değer için 10 deneme ile 100 öğrenme gerçekleştirdiğini ve ne kadar sürdüğünü ekrana yazdırdı.
#Öğrenme süreci bitti

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.9s finished
C:\Users\batuh\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=3000, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.0001, 0.009, 0.01, 0.05, 0.09, 1, 5, 10, 25,
                               100]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='recall', verbose=1)

In [13]:
#Bulduğumuz en iyi değerleri yazdıralım.
best_parameters = grid_model.best_params_
print(best_parameters)

{'C': 1}


In [14]:
y_pred1 = grid_model.predict(X_test_std)
print("Test Performansları;")
print("Doğruluk: ", (accuracy_score(y_test, y_pred1)))
print("Precision: " + str(precision_score(y_test, y_pred1)))
print("Recall: " + str(recall_score(y_test, y_pred1)))
print("F1 puanı: " + str(f1_score(y_test, y_pred1)))
print("\n")
print("Sınıflandırma Tablosu: \n" + str(confusion_matrix(y_test, y_pred1)))

Test Performansları;
Doğruluk:  0.7705627705627706
Precision: 0.6666666666666666
Recall: 0.5675675675675675
F1 puanı: 0.6131386861313869


Sınıflandırma Tablosu: 
[[136  21]
 [ 32  42]]


In [16]:
# Aynı veri setini farklı bir model ile deneyelim.
#2. Karar Destek Makineleri (Support Vector Machines (SVC))
from sklearn.svm import SVC
classifier = SVC(kernel = "rbf", random_state = 0)
classifier.fit(X_train_std, y_train)

y_pred = classifier.predict(X_test_std)
print("Doğruluk: " + str(accuracy_score(y_test, y_pred)))
print("Sınıflandırma Tablosu: \n" + str(confusion_matrix(y_test, y_pred)))

Doğruluk: 0.7445887445887446
Sınıflandırma Tablosu: 
[[135  22]
 [ 37  37]]


In [17]:
#Modeli iyileştirmeye çalışalım
model = SVC()
param_range = [0.0001, 0.001, 0.01, 0.1, 1, 10, 20]
parameters = [{"C": param_range, "kernel": ["linear"]},
              {"C": param_range, "kernel": ["rbf"], "gamma": [0.1, 0.2, 0.4, 0.6, 0.8, 0.9]} ,
              {"C": param_range, "kernel": ["poly"], "degree": [2, 3], "gamma": [0.1, 0.2, 0.4, 0.5, 0.7, 0.9]}]
grid_search = GridSearchCV(model,
                          param_grid = parameters,
                          scoring = "accuracy",
                          cv = 10, verbose = 1,
                          n_jobs = -1)#İşlemcide birden fazla çekirdek varsa ve hepsini kullanmak istiyorsak -1 değeri verilir.
grid_search = grid_search.fit(X_train_std, y_train)

Fitting 10 folds for each of 133 candidates, totalling 1330 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 1330 out of 1330 | elapsed:  3.6min finished
C:\Users\batuh\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [21]:
best_parameters = grid_search.best_params_
print("En iyi parametreler: ", best_parameters)

En iyi parametreler:  {'C': 0.01, 'kernel': 'linear'}


In [22]:
y_pred3 = grid_search.predict(X_test_std)
print("Test Performansları;")
print("Doğruluk: ", (accuracy_score(y_test, y_pred3)))
print("Precision: " + str(precision_score(y_test, y_pred3)))
print("Recall: " + str(recall_score(y_test, y_pred3)))
print("F1 puanı: " + str(f1_score(y_test, y_pred3)))
print("\n")
print("Sınıflandırma Tablosu: \n" + str(confusion_matrix(y_test, y_pred3)))

Test Performansları;
Doğruluk:  0.7662337662337663
Precision: 0.6785714285714286
Recall: 0.5135135135135135
F1 puanı: 0.5846153846153848


Sınıflandırma Tablosu: 
[[139  18]
 [ 36  38]]


In [24]:
#3. Karar ağaçları kullanarak deneyelim.
from sklearn.tree import DecisionTreeClassifier
ml = DecisionTreeClassifier()
ml.fit(x_train, y_train)

y_pred = ml.predict(x_test)
print("Test Verisi için Doğruluk: ", accuracy_score(y_test, y_pred))
print("Test Verisi için Sınıflandırma Tablosu: \n", confusion_matrix(y_test, y_pred))

Test Verisi için Doğruluk:  0.7359307359307359
Test Verisi için Sınıflandırma Tablosu: 
 [[123  34]
 [ 27  47]]


In [29]:
kriter = ["gini", "entropy"]
derinlik = [3, 5, 7, 9, 12]

parametre = dict(criterion= kriter, max_depth = derinlik)

yeni_model = GridSearchCV(estimator = ml, cv = 10, param_grid = parametre, n_jobs = -1, verbose = 1, scoring = "recall")
yeni_model.fit(x_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.2s finished
C:\Users\batuh\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 7, 9, 12]},
             pre_disp

In [30]:
print(yeni_model.best_score_)
print(yeni_model.best_params_)

0.5665147505635597
{'criterion': 'entropy', 'max_depth': 5}


In [31]:
y_tahmin = yeni_model.predict(x_test)
print("Test Performansları;")
print("Doğruluk: ", (accuracy_score(y_test, y_tahmin)))
print("Precision: " + str(precision_score(y_test, y_tahmin)))
print("Recall: " + str(recall_score(y_test, y_tahmin)))
print("F1 puanı: " + str(f1_score(y_test, y_tahmin)))
print("\n")
print("Sınıflandırma Tablosu: \n" + str(confusion_matrix(y_test, y_tahmin)))

Test Performansları;
Doğruluk:  0.7532467532467533
Precision: 0.6164383561643836
Recall: 0.6081081081081081
F1 puanı: 0.6122448979591837


Sınıflandırma Tablosu: 
[[129  28]
 [ 29  45]]


In [36]:
# Rastgele Ormanlar algoritmasını kullanarak deneyelim
# 4. RandomForest
from sklearn.ensemble import RandomForestClassifier
rfr = RandomForestClassifier()

param_grid = {"n_estimators": [150, 200, 250, 300, 350],
              "max_depth": [5, 7, 9, 11, 13 ,15],
              "min_samples_leaf": [3, 5, 7, 9, 11],
              "min_samples_split": [5, 7, 10, 12, 15]}

yeni_model = GridSearchCV(rfr, param_grid, cv = 10, n_jobs = -1, verbose = 1, scoring = "recall")
yeni_model.fit(x_train, y_train)

Fitting 10 folds for each of 750 candidates, totalling 7500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   21.9s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   51.4s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:  9.8min
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 7500 out of 7500 | elapsed: 14.9min finished
C:\Users\batuh\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarni

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             ii

In [37]:
print(yeni_model.best_score_)
print(yeni_model.best_params_)

0.5871263353915515
{'max_depth': 13, 'min_samples_leaf': 3, 'min_samples_split': 7, 'n_estimators': 200}


In [38]:
y_tahmin = yeni_model.predict(x_test)

print("Test Performansları;")
print("Doğruluk: ", (accuracy_score(y_test, y_tahmin)))
print("Precision: " + str(precision_score(y_test, y_tahmin)))
print("Recall: " + str(recall_score(y_test, y_tahmin)))
print("F1 puanı: " + str(f1_score(y_test, y_tahmin)))
print("\n")
print("Sınıflandırma Tablosu: \n" + str(confusion_matrix(y_test, y_tahmin)))

Test Performansları;
Doğruluk:  0.7878787878787878
Precision: 0.7272727272727273
Recall: 0.5405405405405406
F1 puanı: 0.6201550387596899


Sınıflandırma Tablosu: 
[[142  15]
 [ 34  40]]
